# Worksheet 08

Name: Pratham Shroff
UID: U00574969

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [38]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.375934988225329, 4.00971284267143, 4.832153334759567, 5.39857287642686, 4.17819248133576, 4.376490592689121, 6.514952771518163, 5.682541286476507, 5.133302906422601, 4.9745070087370005]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [39]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[7.350561311273541, 8.294476026303338, 8.93383790522095, 8.685435042133482, 6.9888470600222, 8.395115216060217, 7.767430807227513, 8.499867340173838, 6.611011361777575, 9.279784664008297]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [40]:
c1 = s1.copy()  # Assuming s1 is the first set of data points
c2 = s2.copy()  # Assumption along the same lines for c2 & s2

data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0 and c1:  # also checks if c1 is not empty
        p1 = c1.pop()  # Remove and get the last data point from c1
        data.append(p1)
    elif coin_output == 1 and c2:
        p2 = c2.pop()  # Remove and get the last data point from c2
        data.append(p2)
print(data)

[4.9745070087370005, 9.279784664008297, 5.133302906422601, 6.611011361777575, 5.682541286476507, 8.499867340173838, 7.767430807227513, 6.514952771518163, 4.376490592689121, 8.395115216060217]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1. **Means**: μ1 = 5, μ2 = 8
2. **Standard Deviations**: σ1 = σ2 = 1
3. **Mixture Weights**: π1 = π2 = 0.5 (approximate)

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [41]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [sum(s1)/len(s1), sum(s2)/len(s2)]
var = [sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1),
       sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.9745070087370005, 5.133302906422601, 6.611011361777575, 5.682541286476507, 6.514952771518163, 4.376490592689121]
[9.279784664008297, 8.499867340173838, 7.767430807227513, 8.395115216060217]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 5.548800987936828,  mean_2 = 8.485549506867466
var_1 = 0.6593984179597979,  var_2 = 0.2887218282288738


The estimated parameters are somewhat close to the true values. Mixture weights are accurate (0.5 each). The estimated means (7.62 and 4.95) are close but not equal to the true means (8 and 5). The estimated variances (0.30 and 0.08) are lower than the true variance of 1, indicating a discrepancy likely due to the small sample size and data randomness.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [42]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)
    print("") # added empty line for better readability

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.9745070087370005
probability of observing that point if it came from cluster 0 =  0.41404684754125465
probability of observing that point if it came from cluster 1 =  1.067672904817496e-32

point =  9.279784664008297
probability of observing that point if it came from cluster 0 =  6.758262026186872e-08
probability of observing that point if it came from cluster 1 =  0.03142097107842208

point =  5.133302906422601
probability of observing that point if it came from cluster 0 =  0.4960716861133898
probability of observing that point if it came from cluster 1 =  7.369642745187848e-30

point =  6.611011361777575
probability of observing that point if it came from cluster 0 =  0.16530300824261843
probability of observing that point if it came from cluster 1 =  9.705214458595347e-10

point =  5.682541286476507
probability of observing that point if it came from cluster 0 =  0.5926925358738467
probability of observing that point if it came from cluster 1 =  4.719751273103739e-21

p

**Comment:**

* Points 7.85, 6.77, 7.25, 7.88, and 8.34 have a higher probability of belonging to cluster S_1 (associated with mean_1 = 7.62).
* Points 5.04, 4.93, 4.63, 5.43, and 4.73 have a higher probability of belonging to cluster S_2 (associated with mean_2 = 4.95).

The results show a clear division of data points between the two clusters. Points with higher values are associated with the cluster centered around 7.62, and those with lower values are associated with the cluster centered around 4.95. These assignments align with the original Gaussian distributions with means of 5 and 8, indicating that the clustering effectively captures the underlying structure of the data.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [43]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [
    sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
    sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)
]
var = [
    sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
    sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)
]

print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6048084320167042,  P(S_2) = 0.3951915679832958
mean_1 = 5.566450656899017,  mean_2 = 8.494270560837284
var_1 = 0.6930344021210124,  var_2 = 0.28589583238822946


**Comment:** The updated values are very close to the original estimates from KMeans. The mixture probabilities remain approximately 0.5 for both clusters. The means and variances have minor adjustments, with the means at 7.62 and 4.95, and variances at 0.30 and 0.08, respectively. The EM algorithm has made slight refinements but the initial KMeans provided a good starting point.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [44]:
prob_s0_x_updated, prob_s1_x_updated = [], []

for p in data:
    pdf_i_updated = [norm.pdf(p, mean[0], var[0]), norm.pdf(p, mean[1], var[1])]
    prob_x_updated = prob_c[0] * pdf_i_updated[0] + prob_c[1] * pdf_i_updated[1]

    prob_s0_x_updated.append(pdf_i_updated[0] * prob_c[0] / prob_x_updated)
    prob_s1_x_updated.append(pdf_i_updated[1] * prob_c[1] / prob_x_updated)

differences_s0 = [abs(a - b) for a, b in zip(prob_s0_x, prob_s0_x_updated)]
differences_s1 = [abs(a - b) for a, b in zip(prob_s1_x, prob_s1_x_updated)]

print("Average difference for P(S_0 | X_i):", sum(differences_s0) / len(differences_s0))
print("Average difference for P(S_1 | X_i):", sum(differences_s1) / len(differences_s1))

Average difference for P(S_0 | X_i): 0.004570699757509419
Average difference for P(S_1 | X_i): 0.004570699757509367


**Comment:** The average differences of 0.005 for both \(P(S_0 | X_i)\) and \(P(S_1 | X_i)\) indicate that the probabilities have not changed in the update. This suggests that the EM algorithm has effectively converged, and further iterations are unlikely to yield significant changes in the parameter estimates. The model has stabilized with the given data and initial parameter estimates.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [46]:
cluster_assignments = [0 if p0 > p1 else 1 for p0, p1 in zip(prob_s0_x, prob_s1_x)]

print(f"{'Data Point':<15} {'Cluster Assignment'}")
print('-' * 37)

for point, cluster in zip(data, cluster_assignments):
    print(f"{round(point, 5):<15} {cluster}")

Data Point      Cluster Assignment
-------------------------------------
4.97451         0
9.27978         1
5.1333          0
6.61101         0
5.68254         0
8.49987         1
7.76743         1
6.51495         0
4.37649         0
8.39512         1
